In [1]:
import re
import numpy as np
import pandas as pd

from tqdm import tqdm
from datasets import load_dataset
from datasketch import MinHash, MinHashLSH

## 텍스트 데이터에서 유사한 질문들을 찾아내고 중복을 제거하는 작업을 수행

### 편집거리(Edit Distance)와 관련된 개념을 MinHash LSH 알고리즘을 통해 구현한다.

In [2]:
dataset = load_dataset("DopeorNope/New_DPO_dataset")

In [3]:
train_df = pd.DataFrame(dataset['train'])
print(len(train_df))
print(train_df.head())

7576
                                            question  \
0           여자가 유니폼을 입었다.\n     다음 원인에 적절한 답은 무엇일까요?   
1        담배를 재떨이에 툭툭 털었다.\n     다음 결과에 적절한 답은 무엇일까요?   
2  다음 주어진 concept set의 단어를 모두 사용하여 만든 문장들 중 가장 적절...   
3  다음 주어진 concept set의 단어를 모두 사용하여 만든 문장들 중 가장 적절...   
4               현금이 필요하다.\n     다음 결과에 적절한 답은 무엇일까요?   

                           response_j                 response_k        id_lst  
0                      여자의 근무시간이 되었다.             여자의 퇴근시간이 되었다.     Hellaswag  
1                          담뱃재가 떨어졌다.                  재떨이가 깨졌다.     Hellaswag  
2    3. 이 자전거는 올해 처음으로 십억 원의 매출을 올렸다.    1. 매출 분석을 위해 데이터를 수집했다.  Kocommon_Gen  
3  2. 벼슬을 돈 주고 살 수 있는 세상에서 양반이 살고 싶다.  4. 양반들은 보통 많은 돈을 가지고 있었다.  Kocommon_Gen  
4                             은행에 갔다.                    지갑을 샀다.     Hellaswag  


In [4]:
train_df.loc[2]['question']

'다음 주어진 concept set의 단어를 모두 사용하여 만든 문장들 중 가장 적절한 것을 고르세요.\nconcept set: {매출,자전거,올리다}\n1. 매출 분석을 위해 데이터를 수집했다.\n2. 가격을 올리다 보니 고객이 줄었다.\n3. 이 자전거는 올해 처음으로 십억 원의 매출을 올렸다.\n4. 자전거 판매가 감소했다.'

In [5]:
train_df['question'].value_counts()

question
숨이 찼다.\n     다음 원인에 적절한 답은 무엇일까요?                                                                                                                                                                                        3
선풍기를 틀었다.\n     다음 결과에 적절한 답은 무엇일까요?                                                                                                                                                                                     3
가글을 했다.\n     다음 결과에 적절한 답은 무엇일까요?                                                                                                                                                                                       3
배탈이 났다.\n     다음 원인에 적절한 답은 무엇일까요?                                                                                                                                                                                       3
눈이 펑펑 내렸다.\n     다음 결과에 적절한 답은 무엇일까요?                                                                              

### 완전히 동일한 질문 찾기

In [6]:
duplicate_questions = train_df['question'][train_df['question'].duplicated(keep=False)]
duplicate_question_counts = duplicate_questions.value_counts()

In [7]:
len(duplicate_questions)

182

In [8]:
duplicate_questions=duplicate_questions.reset_index(drop=True)
for i in range(len(duplicate_questions)):
    print(duplicate_questions[i])

강아지의 발이 더러워졌다.
     다음 원인에 적절한 답은 무엇일까요?
소방대원들이 출동했다.
     다음 원인에 적절한 답은 무엇일까요?
비행기가 난기류를 만났다.
     다음 결과에 적절한 답은 무엇일까요?
잠긴 문이 열렸다.
     다음 원인에 적절한 답은 무엇일까요?
시력이 나빠졌다.
     다음 결과에 적절한 답은 무엇일까요?
비가 많이 내렸다.
     다음 결과에 적절한 답은 무엇일까요?
노트북에 물을 쏟았다.
     다음 결과에 적절한 답은 무엇일까요?
양산을 썼다.
     다음 원인에 적절한 답은 무엇일까요?
휴대폰을 떨어뜨렸다.
     다음 결과에 적절한 답은 무엇일까요?
눈사람을 만들었다.
     다음 원인에 적절한 답은 무엇일까요?
감기에 걸렸다.
     다음 결과에 적절한 답은 무엇일까요?
손이 끈적거린다.
     다음 원인에 적절한 답은 무엇일까요?
몸이 따뜻해졌다.
     다음 원인에 적절한 답은 무엇일까요?
손톱을 깎았다.
     다음 원인에 적절한 답은 무엇일까요?
양말에 구멍이 났다.
     다음 원인에 적절한 답은 무엇일까요?
풍선이 터졌다.
     다음 원인에 적절한 답은 무엇일까요?
감을 말렸다.
     다음 결과에 적절한 답은 무엇일까요?
손목이 아프다.
     다음 원인에 적절한 답은 무엇일까요?
눈이 펑펑 내렸다.
     다음 결과에 적절한 답은 무엇일까요?
따뜻한 물에 얼음을 넣었다.
     다음 결과에 적절한 답은 무엇일까요?
땀이 난다.
     다음 원인에 적절한 답은 무엇일까요?
휴지에 코를 풀었다.
     다음 결과에 적절한 답은 무엇일까요?
후라이팬에 기름을 둘렀다.
     다음 결과에 적절한 답은 무엇일까요?
감자가 으깨졌다.
     다음 원인에 적절한 답은 무엇일까요?
집에 도둑이 들었다.
     다음 결과에 적절한 답은 무엇일까요?
후라이팬에 기름을 둘렀다.
     다음 결과에 적절한 답은 무엇일까요?
휴지에 코를 풀었다.
     다음 결과에 적절한 답은 

In [9]:
# MinHash LSH 설정
SIMILARITY_THRESHOLD = 0.8
NUM_PERMS = 128 ## 256
SHINGLE_SIZE = 4 ## 5

In [10]:
def preprocess(string, maxlen=500):
    return ''.join(e for e in string[:maxlen] if e.isalnum()).lower()

def _shingle(string, shingle_size=4):
    return {string[i : i + shingle_size] for i in range(len(string) - shingle_size + 1)}

### MinHash LSH
- 편집 거리는 두 문자열이 얼마나 다른지 나타내는 척도.
- MinHash LSH는 대규모 데이터에서 유사한 항목을 효율적으로 찾는 알고리즘
- SIMILARITY_THRESHOLD: 얼마나 유사해야 중복으로 판단할지 결정
- SHINGLE_SIZE: 텍스트를 몇 글자씩 나눠서 비교할지

In [11]:
lsh = MinHashLSH(threshold=SIMILARITY_THRESHOLD, num_perm=NUM_PERMS)

In [12]:
minhashes = {}
for i, doc in tqdm(enumerate(train_df['question'])):
    shingles = _shingle(preprocess(doc), shingle_size=SHINGLE_SIZE)
    minhash = MinHash(num_perm=NUM_PERMS)
    for shingle in shingles:
        minhash.update(shingle.encode('utf8'))
    lsh.insert(f'doc_{i}', minhash)
    minhashes[f'doc_{i}'] = minhash

7576it [00:06, 1205.55it/s]


In [14]:
to_remove = set()

for i, minhash in minhashes.items():
    dups = lsh.query(minhash)
    dups = [int(dup.split('_')[1]) for dup in dups if int(dup.split('_')[1]) > int(i.split('_')[1])]  
    to_remove.update(dups)

print(len(to_remove))

487


In [15]:
train_df_filtered = train_df.drop(list(to_remove))
removed_df = train_df.loc[list(to_remove)]
removed_df=removed_df.reset_index(drop=True)

In [16]:
question_to_check='발에 물집이 생겼다.'
matched_rows = removed_df[removed_df['question'].str.contains(re.escape(question_to_check), regex=True)]
print(matched_rows)

                                   question         response_j  \
176  발에 물집이 생겼다.\n     다음 원인에 적절한 답은 무엇일까요?  하이힐을 하루종일 신고 있었다.   
364  발에 물집이 생겼다.\n     다음 원인에 적절한 답은 무엇일까요?  발에 맞지 않는 구두를 신었다.   

            response_k     id_lst  
176      수영장에 수영하러 갔다.  Hellaswag  
364  발에 딱 맞는 운동화를 신었다.  Hellaswag  


In [18]:
print(train_df_filtered.loc[2]['question'])

다음 주어진 concept set의 단어를 모두 사용하여 만든 문장들 중 가장 적절한 것을 고르세요.
concept set: {매출,자전거,올리다}
1. 매출 분석을 위해 데이터를 수집했다.
2. 가격을 올리다 보니 고객이 줄었다.
3. 이 자전거는 올해 처음으로 십억 원의 매출을 올렸다.
4. 자전거 판매가 감소했다.


In [19]:
print(train_df_filtered.loc[3]['question'])

다음 주어진 concept set의 단어를 모두 사용하여 만든 문장들 중 가장 적절한 것을 고르세요.
concept set: {사다,살다,양반,세상,주다,돈,벼슬}
1. 사람들은 힘들게 벼슬을 하러 세상 밖으로 나간다.
2. 벼슬을 돈 주고 살 수 있는 세상에서 양반이 살고 싶다.
3. 사다리를 타고 올라가면 새로운 세상을 볼 수 있다.
4. 양반들은 보통 많은 돈을 가지고 있었다.


In [20]:
print(train_df_filtered.loc[6]['question'])

다음 주어진 concept set의 단어를 모두 사용하여 만든 문장들 중 가장 적절한 것을 고르세요.
concept set: {청취하다,라디오,회견,기자,안,차,우리}
1. 기자들이 회견에서 질문했다.
2. 우리 집 안에는 차가 없다.
3. 우리는 차 안에서 이번 기자 회견을 라디오로 청취했다.
4. 라디오에서 새로운 노래가 방송되었다.


In [21]:
train_df_filtered.to_csv('train_filtered.csv',index=False)